In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
from hand_analysis import process_poker_hand, save_to_csv
from poker_app.pypokergui.engine.card import processed_hand, percentage_table

In [11]:
# Mapowanie pozycji i akcji
position_map = {'EP': 0, 'MP': 1, 'CO': 2, 'BTN': 3, 'SB': 4, 'BB': 5}
action_map = {'F': 0, 'R': 1}
FILES_PATH = 'hh/*.txt'
hands = process_poker_hand(FILES_PATH)
save_to_csv(hands)

In [12]:
df = pd.read_csv('poker_hand.csv')
df = df[~df['preflop_action'].isin(['X', 'C', 'B'])]  # Usuwamy niechciane akcje
df['position_processed'] = df['position'].map(position_map)  # Mapowanie pozycji
df = df.dropna(subset=['position_processed'])  # Usuwanie wierszy z brakującymi pozycjami
df['hole_cards'] = df['hole_cards'].apply(lambda x: eval(x) if isinstance(x, str) else x)  # Przetwarzanie kart
df['preflop_action_processed'] = df['preflop_action'].map(action_map)
df['preflop_action_processed'] = df['preflop_action_processed'].fillna(0) 



In [16]:
sample_hand = [('As', '2d'), 'BTN']
hand_strength = processed_hand(sample_hand[0], percentage_table)
df_filtered = df[
    df['hole_cards'].apply(lambda x: processed_hand(x, percentage_table) <= hand_strength)
    ]
df_filtered.to_csv('filtered_poker_data.csv', index=False)
X_filtered = pd.DataFrame({
    'hand_strength': df_filtered['hole_cards'].apply(lambda hand: processed_hand(hand, percentage_table)),
    'position': df_filtered['position_processed']
})
y_filtered = df_filtered['preflop_action_processed']
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight= 'balanced')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
sample_hand_processed = pd.DataFrame(
    [[hand_strength, position_map[sample_hand[1]]]],
    columns=['hand_strength', 'position'] 
)
probabilities = np.round(model.predict_proba(sample_hand_processed), 2)
decision = model.predict(sample_hand_processed)


In [17]:
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
TP = cm[1, 1]
TN = cm[0, 0]
FP = cm[0, 1]
FN = cm[1, 0]
sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)
